In [ ]:
import pm4py
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
def import_xes(file_path):
    log = pm4py.read_xes(file_path)
    event_log = pm4py.convert_to_dataframe(log)

    return event_log

event_log = import_xes("/Users/6706363/Downloads/BPI_Challenge_2019.xes")


In [ ]:
df = event_log[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]

df = df.sort_values(by=['org:resource', 'time:timestamp'])

df.head(n=20)

In [ ]:
# Count unique resources
unique_resources = df['org:resource'].nunique()

print(f"Number of unique resources: {unique_resources}")

In [ ]:
# Group by 'org:resource' and count how many activities each resource has performed
activity_counts = df.groupby('org:resource')['concept:name'].count()

# Calculate average activity sequence length per resource
average_activity_length = activity_counts.mean()

print(f"Average activity sequence length per resource: {average_activity_length}")

In [ ]:
# Group by resource and get list of activities performed in order
resource_activity_sequences = df.sort_values('time:timestamp').groupby('org:resource')['concept:name'].apply(list)

# Function to count repetitions in a sequence
def count_repetitions(activity_list):
    from collections import Counter
    counts = Counter(activity_list)
    repetitions = sum(count - 1 for count in counts.values() if count > 1)
    return repetitions

# Apply repetition count to each resource's sequence
repetition_counts = resource_activity_sequences.apply(count_repetitions)

# Average repetition count per resource
average_repetitions = repetition_counts.mean()

print(f"Average activity repetition per resource: {average_repetitions}")

In [ ]:
import pandas as pd
import numpy as np

# Get the relative frequency of activities per resource
activity_counts = df.groupby(['org:resource', 'concept:name']).size()
total_counts = activity_counts.groupby('org:resource').transform('sum')
probabilities = activity_counts / total_counts

# Compute entropy per resource
entropy = probabilities.groupby('org:resource').apply(lambda x: -np.sum(x * np.log2(x)))

# Normalize by max entropy possible (log2 of number of unique activities)
num_unique_acts = df.groupby('org:resource')['concept:name'].nunique()
max_entropy = np.log2(num_unique_acts)

# Avoid division by zero (when only 1 unique activity)
max_entropy_replaced = max_entropy.replace(0, np.nan)
normalized_entropy = entropy / max_entropy_replaced

# Invert to get specialization score
specialization_score = 1 - normalized_entropy

# Treat resources with only 1 activity as fully specialized
specialization_score = specialization_score.fillna(1.0)
normalized_entropy = normalized_entropy.fillna(0.0)

# Combine into DataFrame
specialization_df = pd.DataFrame({
    'entropy': entropy,
    'normalized_entropy': normalized_entropy,
    'specialization_score': specialization_score
})

print(specialization_df)

# Calculate and print average specialization
average_specialization = specialization_score.mean()
print(f"Average specialization across all resources: {average_specialization:.4f}")
